In [379]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [380]:
dataset = pd.read_csv("historicoMaterias.csv")

In [381]:
dataset

,data,url_noticia,url_noticia_curto,titulo,conteudo_noticia,assunto
0,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/m...,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do...",economia
1,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...,economia
2,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/t...,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T...",economia
3,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/g...,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...,economia
4,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...,economia
...,...,...,...,...,...,...
10104,2018-12-22,https://web.archive.org/web/20181222014135/htt...,https://g1.globo.com/economia/tecnologia/notic...,Brasil ganha 10 milhões de internautas em 1 an...,"Em apenas 1 ano, o número de internautas no B...",tecnologia
10105,2018-12-27,https://web.archive.org/web/20181227111959/htt...,https://g1.globo.com/economia/tecnologia/notic...,Como a Coreia do Norte quer usar alta tecnolog...,A Coreia do Norte costuma ostentar seu aparat...,tecnologia
10106,2018-12-27,https://web.archive.org/web/20181227111959/htt...,https://g1.globo.com/economia/tecnologia/notic...,Venda de computadores desacelera por eleições ...,As vendas de computadores no Brasil desaceler...,tecnologia
10107,2018-12-28,https://web.archive.org/web/20181228065113/htt...,https://g1.globo.com/economia/tecnologia/notic...,"Instagram passa a mostrar posts na horizontal,...",O Instagram liberou na tarde desta quinta-fei...,tecnologia


Dropando colunas desncessárias para o objetivo da rede neural

In [382]:
dataset = dataset.drop(["url_noticia", "url_noticia_curto", "assunto", "data"], axis=1)

In [383]:
dataset

,titulo,conteudo_noticia
0,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do..."
1,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...
2,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T..."
3,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...
4,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...
...,...,...
10104,Brasil ganha 10 milhões de internautas em 1 an...,"Em apenas 1 ano, o número de internautas no B..."
10105,Como a Coreia do Norte quer usar alta tecnolog...,A Coreia do Norte costuma ostentar seu aparat...
10106,Venda de computadores desacelera por eleições ...,As vendas de computadores no Brasil desaceler...
10107,"Instagram passa a mostrar posts na horizontal,...",O Instagram liberou na tarde desta quinta-fei...


Como é possível observar na tabela acima, o banco de dados dos textos, ainda possuem alguns caracteres restritos, como /n, /t, /xa0 e etc. Neste caso, estarei tratando para que estes sejam removidos todos do banco de dados

In [384]:
x = 0

for string in dataset["conteudo_noticia"]:
    string = str(string).replace("\n", " ")
    string = str(string).replace("\t", " ")
    string = str(string).replace("\xa0", " ")
    dataset.iloc[x]["conteudo_noticia"] = string
    x+=1


Encurtamento prático

In [385]:
dataset = dataset.iloc[1:100]

In [386]:
dataset

,titulo,conteudo_noticia
1,"Para Dilma, é 'apressada' a tese de que emerge...",A presidente Dilma Rousseff afirmou nesta se...
2,"Temos sido capazes de reduzir a inflação', diz...","O presidente do Banco Central, Alexandre Tom..."
3,Governo argentino autoriza compra de dólares a...,O governo argentino anunciou nesta sexta-fei...
4,Petrobras desiste de parte da área de Bem-te-v...,A Petrobras afirmou que propôs à Agência Nac...
5,"Consórcio deverá investir R$ 5,1 bilhões em li...",Vencedor do leilão da linha de transmissão d...
...,...,...
95,São Paulo goleia Auto Esporte e assume lideran...,O São Paulo se redimiu da estreia decepcionant...
96,"Com 'portunhol' de Dinamite, Vasco apresenta g...",Em alto astral no primeiro dia da reapresentaç...
97,Irmãos de tatuagem': Neymar imita 'Tudo passa'...,"Antes do Natal, o atacante Neymar, do Barcelon..."
98,R10 receberia bônus de R$ 65 mil por jogo no B...,A proposta final do Besiktas já está nas mãos ...


Para que a tokenization seja efetiva, preciso tokenizar o texto todo existente no meu dataset. Este banco de dados possuí blocos de texto, ao invés de ser um texto corrido completo, neste caso, é por esta razão que eu somo o texto todo para então passar o tokenizador. Se eu passar ele individualmente em cada bloco de texto, cada linha do meu dataset teria uma tokenização diferente, ou seja, a mesma palavra em diferentes blocos de texto teria 2 tokens

In [387]:
corpus = ''

for string in dataset["conteudo_noticia"]:
    corpus += string

In [388]:
for string in dataset["titulo"]:
    corpus += string

In [389]:
corpus = corpus.lower().split("\n")

In [390]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

Quantidade total de palavras detectadas no texto

In [391]:
len(tokenizer.word_index)

7392

Nesta parte, eu pego o dataset original, e vou converte-lo para tokens

In [392]:
for x in range(len(dataset["conteudo_noticia"])):
    dataset["conteudo_noticia"].iloc[x] = tokenizer.texts_to_sequences([dataset["conteudo_noticia"].iloc[x]])[0]

C:\Users\migue\AppData\Local\Temp\ipykernel_10052\1525063593.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["conteudo_noticia"].iloc[x] = tokenizer.texts_to_sequences([dataset["conteudo_noticia"].iloc[x]])[0]


In [393]:
for x in range(len(dataset["titulo"])):
    dataset["titulo"].iloc[x] = tokenizer.texts_to_sequences([dataset["titulo"].iloc[x]])[0]

C:\Users\migue\AppData\Local\Temp\ipykernel_10052\2710351441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["titulo"].iloc[x] = tokenizer.texts_to_sequences([dataset["titulo"].iloc[x]])[0]


In [394]:
dataset

,titulo,conteudo_noticia
1,"[9, 175, 15, 7235, 3, 1412, 1, 6, 498, 7236, 1...","[3, 54, 175, 459, 119, 55, 154, 42, 272, 241, ..."
2,"[248, 245, 2412, 1, 770, 3, 7238, 93, 616, 7, ...","[2, 54, 4, 85, 115, 1194, 616, 68, 55, 154, 42..."
3,"[45, 700, 7240, 431, 1, 779, 109, 94, 4, 327]","[2, 45, 700, 557, 55, 154, 42, 272, 6, 3677, 3..."
4,"[189, 7242, 1, 124, 8, 251, 1, 207, 1034, 894,...","[3, 189, 119, 6, 2448, 37, 232, 102, 4, 460, 1..."
5,"[1040, 562, 790, 36, 32, 26, 33, 7, 504, 1, 71...","[1838, 4, 710, 8, 504, 1, 623, 1, 711, 901, 14..."
...,...,...
95,"[51, 107, 7380, 1407, 675, 5, 3572, 1601, 4, 1...","[2, 51, 107, 25, 7004, 8, 866, 7005, 12, 195, ..."
96,"[11, 7382, 1, 1782, 532, 2603, 3544, 5, 7383, ...","[7, 477, 7059, 10, 111, 117, 8, 7060, 4, 7061,..."
97,"[3553, 1, 7385, 1172, 7386, 7387, 7388, 1, 140...","[220, 4, 2324, 2, 318, 1172, 4, 995, 1016, 10,..."
98,"[1773, 2366, 3563, 1, 36, 1113, 136, 17, 497, ...","[3, 472, 187, 4, 1405, 50, 63, 83, 2983, 1, 57..."


In [395]:
maxTitle = max(len(tokens) for tokens in dataset["titulo"])
maxContent = max(len(tokens) for tokens in dataset["conteudo_noticia"])

In [396]:
for titulo in dataset["titulo"]:
    if len(titulo) > 0:
        for x in range(maxTitle - len(titulo)): 
            titulo.append(0)

In [397]:
for conteudo in dataset["conteudo_noticia"]:
    if len(conteudo) > 0:
        for x in range(maxContent - len(conteudo)): 
            conteudo.append(0)

Escrevo o novo dataset após tokenizado, por questões de segurança

In [398]:
import csv 
with open('historicoMateriasTratado.csv', 'w', newline='') as csvfile:

    header = ["titulo", "conteudo_noticia"]
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0 and len(dataset["titulo"].iloc[x]) != 0):           
            writer.writerow(dataset.iloc[x])

In [399]:
import csv 
with open('tituloTratado.csv', 'w', newline='') as csvfile:

    writer = csv.writer(csvfile, delimiter=',')

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0 and len(dataset["titulo"].iloc[x]) != 0):           
            writer.writerow(dataset["titulo"].iloc[x])

In [400]:
import csv 
with open('conteudoTratado.csv', 'w', newline='') as csvfile:

    writer = csv.writer(csvfile, delimiter=',')

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0 and len(dataset["titulo"].iloc[x]) != 0):           
            writer.writerow(dataset["conteudo_noticia"].iloc[x])

In [371]:
import json

with open("tokenizer.json", "w") as jsonfile:
    json.dump(tokenizer.to_json(), jsonfile)